In [ ]:
import sys, shutil, os, re
CWD = os.getcwd()
OSW = os.path.join(CWD, 'doe_prototypes', 'workflow.osw')
OUTDIR = os.path.dirname(CWD)

We want to create many testcases to be uploaded to Alfalfa, using the Create DOE Prototype Building measure. This will allow a user to select (a) a prototypical building type, (b) a building standards template, and (c) a climate zone.

In order to specify these 3 arguments, we must generate a new archive for each testcase, and in each archive the OpenStudio Workflow (OSW) file will be modified to specify the correct measure arguments.

### Select the arguments

The values you can choose for each argument are copied below:

| Building Type            | Template            | Climate Zone         |
|--------------------------|---------------------|----------------------|
| 'SecondarySchool'        | 'DOE Ref Pre-1980'  | 'ASHRAE 169-2013-1A' |
| 'PrimarySchool'          | 'DOE Ref 1980-2004' | 'ASHRAE 169-2013-2A' |
| 'SmallOffice'            | '90.1-2004'         | 'ASHRAE 169-2013-2B' |
| 'MediumOffice'           | '90.1-2007'         | 'ASHRAE 169-2013-3A' |
| 'LargeOffice'            | '90.1-2010'         | 'ASHRAE 169-2013-3B' |
| 'SmallHotel'             | '90.1-2013'         | 'ASHRAE 169-2013-3C' |
| 'LargeHotel'             | '90.1-2016'         | 'ASHRAE 169-2013-4A' |
| 'Warehouse'              |                     | 'ASHRAE 169-2013-4B' |
| 'RetailStandalone'       |                     | 'ASHRAE 169-2013-4C' |
| 'RetailStripmall'        |                     | 'ASHRAE 169-2013-5A' |
| 'QuickServiceRestaurant' |                     | 'ASHRAE 169-2013-5B' |
| 'FullServiceRestaurant'  |                     | 'ASHRAE 169-2013-6A' |
| 'MidriseApartment'       |                     | 'ASHRAE 169-2013-6B' |
| 'HighriseApartment'      |                     | 'ASHRAE 169-2013-7A' |
| 'Hospital'               |                     | 'ASHRAE 169-2013-8A' |
| 'Outpatient'             |                     |                      |
| 'Laboratory'             |                     |                      |
| 'LargeDataCenterHighITE' |                     |                      |
| 'LargeDataCenterLowITE'  |                     |                      |
| 'SmallDataCenterHighITE' |                     |                      |
| 'SmallDataCenterLowITE'  |                     |                      |


We define a function to create testcases. This function will create a new directory for each testcase, copy the contents of the `doe_prototypes` folder and amend the OSW with the correct arguments before archiving the files and deleting the directory.

In [ ]:
def createTestcase(arguments, debug=False):
    # create new directory
    tdirname = arguments['building_type'] + arguments['template'] + arguments['climate_zone']
    tdirname =re.sub("[^a-zA-Z0-9_]", "_", tdirname,0,re.IGNORECASE)
    TEMP = os.path.join(OUTDIR, tdirname)
    print(TEMP)
    if os.path.exists(TEMP):
        shutil.rmtree(TEMP)
    #os.mkdir(TEMP)
    shutil.copytree(CWD, TEMP)
    tlist = os.listdir(TEMP)
    # remove the existing OSW and this notebook from the new dir
    for item in tlist:
        if item.endswith(".osw") or item.endswith(".ipynb"):
            os.remove(os.path.join(TEMP, item))
    NOSW = os.path.join(TEMP, 'doe_prototypes', 'workflow.osw')
    # read the base OSW and modify it, write to new OSW
    with open(OSW, 'r') as origosw:
        with open(NOSW, 'w') as newosw:
            for line in origosw.readlines():
                # If we find one of these keywords, replace them with the correct arguments. Else, the line is copied as-is.
                if '##DOEBLDGTYPE##' in line:
                    line = line.replace('##DOEBLDGTYPE##', arguments['building_type'])
                elif '##TEMPLATE##' in line:
                    line = line.replace('##TEMPLATE##', arguments['template'])
                elif '##CLIMATEZONE##' in line:
                    line = line.replace('##CLIMATEZONE##', arguments['climate_zone'])
                newosw.writelines(line)
    # zip it
    shutil.make_archive(TEMP, 'zip', TEMP)
    # clean it
    shutil.rmtree(TEMP)

Here is an example to generate a single testcase:

In [ ]:
createTestcase({
    'building_type': 'PrimarySchool',
    'climate_zone': 'ASHRAE 169-2013-8A',
    'template': '90.1-2016'
})

And here for many buildings:

In [ ]:
for building in ['PrimarySchool', 'SecondarySchool']:
    for climatezone in ['ASHRAE 169-2013-4A', 'ASHRAE 169-2013-6B']:
        for template in ['90.1-2007', '90.1-2013', '90.1-2016']:
            createTestcase({
                'building_type': building,
                'climate_zone': climatezone,
                'template': template
            })